In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib
import cv2
import seaborn as sns

In [ ]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')

In [ ]:
import os
os.getcwd()

In [ ]:
data_dir = 'D:\Brain Cancer\Dataset\Brain_Cancer raw MRI data\Brain_Cancer'
os.listdir(data_dir)

In [ ]:
tf.random.set_seed(42)

train_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'training', seed = 1, shuffle = True, batch_size = 32, image_size=(128,128))

test_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'validation', seed = 1, shuffle = True, batch_size = 32, image_size=(128,128))

In [ ]:
filenames = pathlib.Path(data_dir)
for label in train_data.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_data.cardinality().numpy(),  test_data.cardinality().numpy()

In [ ]:
train_set = train_data.take(152)
val_set = train_data.skip(152)

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy()

In [ ]:
# import necessary libraries
import os
import matplotlib.pyplot as plt
import random

# Set the path to save the plot as PDF
save_path = r'D:\Brain Cancer\PDF\random_train_images.pdf'

# Create the plot
plt.figure(figsize=(15, 15))
for images, labels in train_set.take(1):
    for i in range(5):
        index = random.randint(0, len(images) - 1)
        ax = plt.subplot(1, 5, i + 1)
        plt.imshow(images[index].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[index]], color='blue', fontsize=12)
        plt.axis(False)

# Adjust layout to be tight and save as PDF
plt.tight_layout()
plt.savefig(save_path, format='pdf',bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
for images_batch, labels_batch in train_data:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Flatten
from tensorflow.keras import Model, Input

In [ ]:
tf.random.set_seed(42)

# Define the input layer
input_layer = Input(shape=(128, 128, 3))

# Convolutional layers with max pooling
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Flatten the output
x = Flatten()(x)

# Fully connected layers with dropout
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)

# Output layer
output_layer = Dense(3, activation='softmax')(x)

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Print the model summary
model.summary()

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics = 'accuracy')

In [ ]:
history_1 = model.fit(train_data, epochs=300, validation_data=test_data)

In [ ]:
model.summary()

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    #     # Specify the directory to save the PDF
    # save_dir = 'D:\\Brain Cancer\\PDF'
    
    # # Create the directory if it doesn't exist
    # if not os.path.exists(save_dir):
    #     os.makedirs(save_dir)
    
    # # Full path to save the PDF file
    # pdf_path = os.path.join(save_dir, 'CNN_training_curves.pdf')
    
    # # Save the plot as a PDF with tight layout
    # plt.tight_layout()  # Ensure that layout is tight
    # plt.savefig(pdf_path, format='pdf', bbox_inches='tight')  
    # Show the plot
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_1.history))

In [ ]:
X_test, y_test = None, None
for images, labels in test_data:
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
from sklearn import metrics

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
test_score = model.evaluate(test_data, verbose= 1)

print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['brain_glioma', 'brain_menin', 'brain_tumor']
print(classification_report(y_test , y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
plt.figure(figsize = (6,6), dpi = 100)
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, fmt='d', cmap = 'Greens')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
    # Add a title with the model name VGG19
plt.title('Confusion Matrix for CNN')

#     # Specify the directory to save the PDF
# save_dir = 'D:\\Brain Cancer\\PDF'
    
#     # Create the directory if it doesn't exist
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

#     # Full path to save the PDF file (with VGG19 in the filename)
# pdf_path = os.path.join(save_dir, 'CNN_confusion_matrix.pdf')
    
#     # Save the plot as a PDF with tight layout
# plt.tight_layout()  # Ensure that layout is tight
# plt.savefig(pdf_path, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)

In [ ]:
plot_random_predictions(test_data, model)

In [ ]:
# Function to save and display GradCAM
def save_and_display_gradcam(img_path, heatmap, alpha=0.4):
    # Load the original image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))  # Resize image to match model input size

    # Resize heatmap to match the image dimensions
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Apply heatmap on the original image
    superimposed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    # Display the GradCAM visualization using Matplotlib
    plt.figure(figsize=(3, 3))
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title('GradCAM', fontdict={'family': 'Serif', 'weight': 'bold', 'size': 12})
    plt.axis('off')
    plt.tight_layout()
    # Save the figure
    plt.savefig('D:\Balanced Augmented Covid CXR Dataset\CNN_gradcam.pdf')  # Save as pdf format
    plt.show()

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):

    model.layers[-1].activation = None
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
 # make a prediction and visualize grad-cam
def make_prediction_and_visualize_():
    img_path = 'D:\Brain Cancer\Dataset\Brain_Cancer raw MRI data\Brain_Cancer\brain_menin\brain_menin_0005.jpg'

    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256)) #IMG_WIDTH, IMG_HEIGHT
    batch_pred = np.expand_dims(img, 0)

    #rescaled_img = img / 255.0
    #batch_pred = np.expand_dims(rescaled_img, 0)


    last_conv_layer_name = 'max_pooling2d_8'

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(batch_pred, model, last_conv_layer_name)

    save_and_display_gradcam(img_path, heatmap)


make_prediction_and_visualize_()

In [ ]:
# Function to save and display ScoreCAM
def save_and_display_gradcam_plusplus(img_path, heatmap, alpha=0.4):
    # Load the original image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))  # Resize image to match model input size

    # Resize heatmap to match the image dimensions
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Apply heatmap on the original image
    superimposed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    # Display the GradCAM visualization using Matplotlib
    plt.figure(figsize=(3, 3))
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title('GradCAM++', fontdict={'family': 'Serif', 'weight': 'bold', 'size': 12})
    plt.axis('off')
    plt.tight_layout()
    # Save the figure
    plt.savefig('D:\Balanced Augmented Covid CXR Dataset\CNN_gradcamplusplus.pdf')  # Save as pdf format
    plt.show()

In [ ]:
# Function to generate GradCAM++ heatmap
def make_gradcam_plusplus_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    model.layers[-1].activation = None
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_output = preds[:, pred_index]
        conv_output = last_conv_layer_output[0]

    # Get gradients
    grads = tape.gradient(class_output, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads[0], axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]

    # Calculate guided gradients
    guided_grads = tf.cast(last_conv_layer_output > 0, 'float32') * grads[0]

    # Calculate importance weights
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))

    # Generate heatmap
    heatmap = tf.reduce_sum(tf.multiply(weights, last_conv_layer_output), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)  # Normalize

    return heatmap.numpy()

In [ ]:
# Function to make a prediction and visualize GradCAM++
def make_prediction_and_visualize_gradcam_plusplus():
    img_path = 'D:\Balanced Augmented Covid CXR Dataset\Dataset\Chest X-Ray Image\Covid\\35.jpg'


    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))  # Resize image to match model input size

    batch_pred = np.expand_dims(img, 0)

    last_conv_layer_name = 'max_pooling2d_8'

    # Generate GradCAM++ heatmap
    heatmap = make_gradcam_plusplus_heatmap(batch_pred, model, last_conv_layer_name)

    save_and_display_gradcam_plusplus(img_path, heatmap)

make_prediction_and_visualize_gradcam_plusplus()